# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 5:16PM,250766,16,SHL-8776117,"SHL Pharma, LLC",Released
1,Nov 14 2022 5:11PM,250765,19,ACG-2102491861,ACG North America LLC,Released
2,Nov 14 2022 4:57PM,250764,10,CTF0011462-BO,"Citieffe, Inc.",Released
3,Nov 14 2022 4:57PM,250764,10,CTF0011495-BO,"Citieffe, Inc.",Released
4,Nov 14 2022 4:57PM,250764,10,CTF0011503-BO,"Citieffe, Inc.",Released
5,Nov 14 2022 4:40PM,250761,10,MSP215834,Methapharm-G,Released
6,Nov 14 2022 4:09PM,250757,21,RE111422,"NBTY Global, Inc.",Released
7,Nov 14 2022 4:07PM,250755,10,PRF-41854,Bio-PRF,Released
8,Nov 14 2022 4:07PM,250755,10,PRF-41858,Bio-PRF,Released
9,Nov 14 2022 4:07PM,250755,10,PRF-41861,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,250757,Released,1
32,250761,Released,1
33,250764,Released,3
34,250765,Released,1
35,250766,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250757,NaN,NaN,1.0
250761,NaN,NaN,1.0
250764,NaN,NaN,3.0
250765,NaN,NaN,1.0
250766,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250634,1.0,0.0,1.0
250664,15.0,2.0,1.0
250668,21.0,6.0,10.0
250669,0.0,0.0,4.0
250674,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250634,1,0,1
250664,15,2,1
250668,21,6,10
250669,0,0,4
250674,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250634,1,0,1
1,250664,15,2,1
2,250668,21,6,10
3,250669,0,0,4
4,250674,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250634,1,,1
1,250664,15,2,1
2,250668,21,6,10
3,250669,,,4
4,250674,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC"
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc."
5,Nov 14 2022 4:40PM,250761,10,Methapharm-G
6,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc."
7,Nov 14 2022 4:07PM,250755,10,Bio-PRF
36,Nov 14 2022 4:01PM,250751,10,ISDIN Corporation
59,Nov 14 2022 4:01PM,250750,10,ISDIN Corporation
64,Nov 14 2022 3:59PM,250754,10,ISDIN Corporation
69,Nov 14 2022 3:56PM,250747,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC",,,1
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC,,,1
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc.",,,3
3,Nov 14 2022 4:40PM,250761,10,Methapharm-G,,,1
4,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc.",,,1
5,Nov 14 2022 4:07PM,250755,10,Bio-PRF,,,29
6,Nov 14 2022 4:01PM,250751,10,ISDIN Corporation,,,23
7,Nov 14 2022 4:01PM,250750,10,ISDIN Corporation,,,5
8,Nov 14 2022 3:59PM,250754,10,ISDIN Corporation,,,5
9,Nov 14 2022 3:56PM,250747,10,"Snap Medical Industries, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC",1,,
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC,1,,
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc.",3,,
3,Nov 14 2022 4:40PM,250761,10,Methapharm-G,1,,
4,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc.",1,,
5,Nov 14 2022 4:07PM,250755,10,Bio-PRF,29,,
6,Nov 14 2022 4:01PM,250751,10,ISDIN Corporation,23,,
7,Nov 14 2022 4:01PM,250750,10,ISDIN Corporation,5,,
8,Nov 14 2022 3:59PM,250754,10,ISDIN Corporation,5,,
9,Nov 14 2022 3:56PM,250747,10,"Snap Medical Industries, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC",1,,
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC,1,,
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc.",3,,
3,Nov 14 2022 4:40PM,250761,10,Methapharm-G,1,,
4,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC,1.0,NaN,NaN
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc.",3.0,NaN,NaN
3,Nov 14 2022 4:40PM,250761,10,Methapharm-G,1.0,NaN,NaN
4,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 5:16PM,250766,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Nov 14 2022 5:11PM,250765,19,ACG North America LLC,1.0,0.0,0.0
2,Nov 14 2022 4:57PM,250764,10,"Citieffe, Inc.",3.0,0.0,0.0
3,Nov 14 2022 4:40PM,250761,10,Methapharm-G,1.0,0.0,0.0
4,Nov 14 2022 4:09PM,250757,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3510109,148.0,15.0,40.0
15,250664,1.0,2.0,15.0
16,250766,1.0,0.0,0.0
19,1002883,6.0,1.0,23.0
20,250695,1.0,0.0,0.0
21,501431,2.0,0.0,0.0
22,250723,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3510109,148.0,15.0,40.0
1,15,250664,1.0,2.0,15.0
2,16,250766,1.0,0.0,0.0
3,19,1002883,6.0,1.0,23.0
4,20,250695,1.0,0.0,0.0
5,21,501431,2.0,0.0,0.0
6,22,250723,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,148.0,15.0,40.0
1,15,1.0,2.0,15.0
2,16,1.0,0.0,0.0
3,19,6.0,1.0,23.0
4,20,1.0,0.0,0.0
5,21,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,148.0
1,15,Released,1.0
2,16,Released,1.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,40.0,15.0,0.0,23.0,0.0,0.0,0.0
Executing,15.0,2.0,0.0,1.0,0.0,0.0,0.0
Released,148.0,1.0,1.0,6.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,40.0,15.0,0.0,23.0,0.0,0.0,0.0
1,Executing,15.0,2.0,0.0,1.0,0.0,0.0,0.0
2,Released,148.0,1.0,1.0,6.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,40.0,15.0,0.0,23.0,0.0,0.0,0.0
1,Executing,15.0,2.0,0.0,1.0,0.0,0.0,0.0
2,Released,148.0,1.0,1.0,6.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()